In [26]:
import torch
import torch.nn as nn
import torchtext
import pkuseg
import pandas as pd

In [2]:
totalpath = '../data/movie_remiew/train_data.csv'
testpath = '../data/movie_remiew/test_data.csv'

In [3]:
totaldf = pd.read_csv(totalpath)
totaldf.head(3)

,label,comment
0,0,国王的工作就是读几句稿子啊
1,0,小朋友看嫌复杂大朋友看想快进的尴尬人物似曾相识唯一的泪点也是复制黏贴
2,0,一个非常丰富传奇的故事拍得这么浅薄大家是怎么给出五星的好奇


In [4]:
totaldf.label.value_counts()

1    24960
0    19108
Name: label, dtype: int64

In [5]:
totaldf.shape[0]

44068

In [6]:
totaldf = totaldf.sample(frac=1).reset_index(drop=True)

In [7]:
totaldf.head()

,label,comment
0,1,最后一星是给主题曲的相知相恋的两人还是因为时间空间的距离慢慢的开始自己新的生活虽然心里的那个...
1,1,难以描述太好了这里有你要的一切
2,1,当我忘记的时候也想有个人让我回忆起当年的幸福
3,1,看看这蠕动的虫滋溜滋溜高蛋白诶人类无法生存的原因是自己污染了土地自然的进化是为了修复只要有干...
4,0,因为是李安这老男人拍的所以被捧上天了我看着有很多让人反感的地方吴倩莲可真美啊


In [8]:
validdf = totaldf.iloc[:4000]
traindf = totaldf.iloc[4000:]

validdf.to_csv('../data/movie_remiew/validset.csv', index=False)
traindf.to_csv('../data/movie_remiew/trainset.csv', index=False)

In [9]:
testdf = pd.read_csv(testpath)
testdf.head(3)

,comment
0,画面很美但台词实在是太太太矫情不是十几岁少年的正常青涩哪怕故作的沧桑而像没文化的油腻中年对键...
1,这片子好看好看好看好看重要的问题问四遍竟然是前百失望致幻的嗨药是弱者逃避现实的选择你觉得现实...
2,老电影有老电影说不出来的美感即使是近三个小时的电影细节做的仍非精致七武士的故事人类百科


In [10]:
testdf.shape[0]

9999

In [11]:
seg = pkuseg.pkuseg()

In [12]:
TEXT = torchtext.data.Field(init_token='<sos>', eos_token='<eos>', tokenize=seg.cut)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [13]:
train, valid = torchtext.data.TabularDataset.splits(path='../data/movie_remiew/', 
                                                    train='trainset.csv', 
                                                    validation='validset.csv',
                                                    format='csv', 
                                                    skip_header=True,
                                                    fields=[('label', LABEL), 
                                                            ('text', TEXT),])

In [14]:
TEXT.build_vocab(train)

In [15]:
TEXT.vocab.stoi['我']

7

In [19]:
train[0].text

['相当',
 '喜欢',
 '的',
 '电影',
 '可以',
 '说',
 '是',
 '最',
 '喜欢',
 '我',
 '喜欢',
 '那种',
 '慵懒',
 '的',
 '氛围',
 '喜欢',
 '那种',
 '鲜艳',
 '的',
 '色调',
 '喜欢',
 '手风琴',
 '的',
 '声音']

In [20]:
train[0].label

'1'

In [31]:
dir(torchtext.data)

['BPTTIterator',
 'Batch',
 'BucketIterator',
 'Dataset',
 'Example',
 'Field',
 'Iterator',
 'LabelField',
 'NestedField',
 'Pipeline',
 'RawField',
 'ReversibleField',
 'SubwordField',
 'TabularDataset',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'batch',
 'dataset',
 'example',
 'field',
 'get_tokenizer',
 'interleave_keys',
 'iterator',
 'pipeline',
 'pool',
 'utils']

In [25]:
trainiter, validiter =  torchtext.data.BucketIterator.splits((train, valid), batch_sizes=(64, 64))

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embed_dim, class_num):
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, 1)
        self.init_weights()
        
    def init_weights(self):
        self.embedding.data.uniform_(-0.5, 0.5)
        self.fc.weight.data.uniform_(-0.5, 0.5)
        self.fc.bias.data.zero_()
        
    def forward(self, x):
        